<a href="https://colab.research.google.com/github/vsolodkyi/NeuralNetworks_SkillBox/blob/main/module_3/3_9_%D0%A1%D0%BE%D1%85%D1%80%D0%B0%D0%BD%D0%B5%D0%BD%D0%B8%D0%B5_%D0%B8_%D0%B7%D0%B0%D0%B3%D1%80%D1%83%D0%B7%D0%BA%D0%B0_%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D0%B5%D0%B9_%D0%B2_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Сохранение и возобновление обучения в Tensorflow

До этого момента мы с вами работали с "игрушечными" данными и основной целью было понять основные механизмы работы фреймворка, обучения и построения моделей. 

Раньше мы не думали о том, как можно использовать обученную модель в будущем и сохранять параметры, которые мы получили во время обучения. 

Именно об этих возможностях, обязательных в практическом применении сетей, и пойдет речь в этом уроке.

## Датасет

В этом уроке мы будем работать с датасетом Pima Indian Diabetes, в котором приведены данные о состоянии здоровья людей. Помимо физиологических показаний, в данных есть информация о наличии у человека диабета. Это и будет считаться классом.

In [ ]:
# загрузим данные
! wget https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv
import pandas as pd

dataset = pd.read_csv("pima-indians-diabetes.data.csv", header=None)
dataset.columns = ["number_times_pregnant",
"plasma_glucose_concent","blood_pressure",
"skin_thickness", "insulin", "BMI", "diabetes_pedigree", "age", "has_diabetes"]
dataset.head()

--2022-02-03 14:23:19--  https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23278 (23K) [text/plain]
Saving to: ‘pima-indians-diabetes.data.csv’

pima-indians-diabet 100%[===================>]  22.73K  --.-KB/s    in 0.002s  

2022-02-03 14:23:19 (8.95 MB/s) - ‘pima-indians-diabetes.data.csv’ saved [23278/23278]



,number_times_pregnant,plasma_glucose_concent,blood_pressure,skin_thickness,insulin,BMI,diabetes_pedigree,age,has_diabetes
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

np.random.seed(10)
X = dataset.drop(['has_diabetes'], axis=1).values
y = dataset['has_diabetes'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}, Labels ratio: {y_train.mean():.2f}")

Train shape: (614, 8), Test shape: (154, 8), Labels ratio: 0.34


## Сохранение и загрузка моделей

Т.к. параметры нашей модели трансформируются путем оптимизации из случайных -- в полезные для нашей задачи, нам бы хотелось сохранять этот прогресс. Было бы неразумно каждый раз обучать модель заново. Сохранение параметров модели во время обучения называют -- чекпоинтом (checkpoint).

Подходы к сохранению моделей могут варьироваться в зависимости от количества данных и сложности модели. Все они сводятся к правилам по которым решается когда сохранять модель. Приведем самые популярные из таких правил:
*   Сохранение в конце каждой эпохи
*   Сохранение каждые k эпох
*   Сохранение только последней эпохи
*   Сохранение весов при улучшении метрики


In [ ]:
import tensorflow as tf
import numpy as np
#tf.enable_eager_execution()
print(tf.__version__)

2.7.0


Для начала обучим модель и убедимся, что все работает:

In [ ]:
def get_compiled_model():
    """
    Функция возвращает скомпилированную модель для бинарной классификации
    """
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(24, input_dim=8, activation='relu'))
    model.add(tf.keras.layers.Dense(24, activation='relu'))
    model.add(tf.keras.layers.Dense(24, activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

model = get_compiled_model()
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=150, batch_size=64, verbose=1)

Epoch 1/150
10/10 [==============================] - 4s 87ms/step - loss: 3.1077 - accuracy: 0.5668 - val_loss: 1.9139 - val_accuracy: 0.3896
Epoch 2/150
10/10 [==============================] - 0s 20ms/step - loss: 1.0856 - accuracy: 0.5586 - val_loss: 0.7372 - val_accuracy: 0.4935
Epoch 3/150
10/10 [==============================] - 0s 13ms/step - loss: 0.8185 - accuracy: 0.5831 - val_loss: 0.6174 - val_accuracy: 0.6558
Epoch 4/150
10/10 [==============================] - 0s 18ms/step - loss: 0.7399 - accuracy: 0.6140 - val_loss: 0.6092 - val_accuracy: 0.6688
Epoch 5/150
10/10 [==============================] - 0s 27ms/step - loss: 0.6735 - accuracy: 0.6596 - val_loss: 0.6392 - val_accuracy: 0.5779
Epoch 6/150
10/10 [==============================] - 0s 18ms/step - loss: 0.6628 - accuracy: 0.6287 - val_loss: 0.6289 - val_accuracy: 0.6234
Epoch 7/150
10/10 [==============================] - 0s 22ms/step - loss: 0.6186 - accuracy: 0.6759 - val_loss: 0.5853 - val_accuracy: 0.6753
Epoch 

### Сохранение модели

За запись чекпоинтов в keras отвечает **tf.keras.callbacks.ModelCheckpoint**. Как видно из пути к модулю -- он относится к коллбекам (callbacks). Можно смотреть на них как на функции, которые вызывает метод model.fit внутри себя.

То есть каждый раз, когда отработает одна из эпох цикла, будет запускаться функция, которую мы укажем. Бывают коллбеки, которые вызываются на каждой итерации (например [прогресс бар](https://keras.io/callbacks/#progbarlogger), который мы видим при обучении -- это тоже коллбэк). Чуть ниже мы разберем конкретный пример. 

Вернемся к **tf.keras.callbacks.ModelCheckpoint**. 

```
tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', ...)
```

Он имеет один обязательный параметр -- имя чекпоинта. Оно может быть отформатировано с использованием служебных переменных (пример ниже). Из необязательных, но полезных параметров:

*   monitor (str): название метрики за которой нужно "следить"
*   verbose -- печатать ли дополнительную информацию
*   save_best_only (bool): сохранять чекпоинт только после улучшения метрики, указанной в monitor
*   save_weights_only (bool): помимо весов модели можно еще сохранять ее архитектуру и состояние оптимизатора. В примерах мы так и будем делать, если не сказано обратное. По умолчанию -- false.
*   mode (str): если метрику нужно максимизировать -- max, иначе -- min.


**Таким образом, если выбрано save_best_only, то чекпоинты будут сохранятся только после тех эпох, когда произошло улучшение метрики, которую мы указали в monitor. Т.к. понятие "лучше" -- разное для разных метрик, необходимо также указать режим -- max/min (больше -- лучше/меньше -- лучше). Пример таких метрик -- лосс (min) и точность (max)**.

In [ ]:
from pathlib import Path
path = Path("model_1")
path.mkdir(exist_ok=True) # создаем папку на диске
cpt_filename = "{epoch:02d}_checkpoint_{val_loss:.2f}.hdf5"  
cpt_path = str(path / cpt_filename)

checkpoint = tf.keras.callbacks.ModelCheckpoint(cpt_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [ ]:
tf.keras.callbacks.ModelCheckpoint?

In [ ]:
"a={a_val}, b={b_val}".format(a_val=2, b_val=34)

'a=2, b=34'

In [ ]:
cpt_path

'model_1/{epoch:02d}_checkpoint_{val_loss:.2f}.hdf5'

In [ ]:
# внутри цикла имя чекпоинта будет сгенерировано примерно таким образом:
for epoch in range(4):
    print(cpt_path.format(epoch=epoch, val_loss=np.random.rand()))

model_1/00_checkpoint_0.67.hdf5
model_1/01_checkpoint_0.39.hdf5
model_1/02_checkpoint_0.22.hdf5
model_1/03_checkpoint_0.07.hdf5


Мы используем hdf5 -- основной формат для keras. Для Tensorflow -- это ckpt. Сохранять в этом расширении можно аналогичным образом. Подробнее можно прочитать [здесь](https://www.tensorflow.org/tutorials/keras/save_and_restore_models).

In [ ]:
model = get_compiled_model()
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=150, batch_size=64, verbose=0, 
          callbacks=[checkpoint])


Epoch 00001: val_loss improved from inf to 0.70699, saving model to model_1/01_checkpoint_0.71.hdf5

Epoch 00002: val_loss did not improve from 0.70699

Epoch 00003: val_loss improved from 0.70699 to 0.69121, saving model to model_1/03_checkpoint_0.69.hdf5

Epoch 00004: val_loss improved from 0.69121 to 0.62367, saving model to model_1/04_checkpoint_0.62.hdf5

Epoch 00005: val_loss did not improve from 0.62367

Epoch 00006: val_loss did not improve from 0.62367

Epoch 00007: val_loss did not improve from 0.62367

Epoch 00008: val_loss did not improve from 0.62367

Epoch 00009: val_loss did not improve from 0.62367

Epoch 00010: val_loss improved from 0.62367 to 0.59557, saving model to model_1/10_checkpoint_0.60.hdf5

Epoch 00011: val_loss did not improve from 0.59557

Epoch 00012: val_loss did not improve from 0.59557

Epoch 00013: val_loss did not improve from 0.59557

Epoch 00014: val_loss did not improve from 0.59557

Epoch 00015: val_loss did not improve from 0.59557

Epoch 00016

Убедимся ниже, что файлы сохранены:

In [ ]:
!ls "model_1/"

01_checkpoint_0.71.hdf5   123_checkpoint_0.53.hdf5  41_checkpoint_0.57.hdf5
03_checkpoint_0.69.hdf5   25_checkpoint_0.59.hdf5   67_checkpoint_0.56.hdf5
04_checkpoint_0.62.hdf5   31_checkpoint_0.59.hdf5   71_checkpoint_0.56.hdf5
106_checkpoint_0.54.hdf5  38_checkpoint_0.58.hdf5   79_checkpoint_0.56.hdf5
10_checkpoint_0.60.hdf5   39_checkpoint_0.58.hdf5   89_checkpoint_0.55.hdf5


### Загрузка модели

Загрузить модель можно двумя способами. Определить модель и загрузить в нее веса, с использованием *model.load_weights()*. Или tf.keras.models.load_model для автоматического создания модели и загрузки весов (мы можем так сделать т.к. при создании чекпоинтов выставили save_weights_only=False, т.е. сохранили и описание модели).

In [ ]:
model = get_compiled_model() # определим случайно инициализированную модель
loss, acc = model.evaluate(X_test, y_test)
print(f"Accuracy of random model {acc*100 :.2f}%")

5/5 [==============================] - 0s 3ms/step - loss: 1.4367 - accuracy: 0.4156
Accuracy of random model 41.56%


In [ ]:
model.load_weights("model_1/89_checkpoint_0.55.hdf5") # загружаем веса. обратите внимание, модель уже создана!
loss, acc = model.evaluate(X_test, y_test)
print(f"Accuracy of restored model {acc*100 :.2f}%")

5/5 [==============================] - 0s 4ms/step - loss: 0.5480 - accuracy: 0.7532
Accuracy of restored model 75.32%


In [ ]:
# т.к. мы сохраняли и веса и модель, то мы можем загрузить модель не определяя ее
model = tf.keras.models.load_model("model_1/89_checkpoint_0.55.hdf5")
loss, acc = model.evaluate(X_test, y_test)
print(f"Accuracy of restored model {acc*100 :.2f}%")

5/5 [==============================] - 0s 4ms/step - loss: 0.5480 - accuracy: 0.7532
Accuracy of restored model 75.32%


### Сохранение модели в один чекпоинт
Если не форматировать название файла, то все чекпоинты будут записаны в него:

In [ ]:
path = Path("model_2")
path.mkdir(exist_ok=True)
cpt_filename = "checkpoint_best.hdf5"
cpt_path =str(path / cpt_filename)

checkpoint = tf.keras.callbacks.ModelCheckpoint(cpt_path, monitor='val_loss', verbose=1, save_best_only=True, mode='max')

model = get_compiled_model()
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=150, batch_size=64, verbose=0, 
          callbacks=[checkpoint])


Epoch 00001: val_loss improved from -inf to 1.18403, saving model to model_2/checkpoint_best.hdf5

Epoch 00002: val_loss did not improve from 1.18403

Epoch 00003: val_loss did not improve from 1.18403

Epoch 00004: val_loss did not improve from 1.18403

Epoch 00005: val_loss did not improve from 1.18403

Epoch 00006: val_loss did not improve from 1.18403

Epoch 00007: val_loss did not improve from 1.18403

Epoch 00008: val_loss did not improve from 1.18403

Epoch 00009: val_loss did not improve from 1.18403

Epoch 00010: val_loss did not improve from 1.18403

Epoch 00011: val_loss did not improve from 1.18403

Epoch 00012: val_loss did not improve from 1.18403

Epoch 00013: val_loss did not improve from 1.18403

Epoch 00014: val_loss did not improve from 1.18403

Epoch 00015: val_loss did not improve from 1.18403

Epoch 00016: val_loss did not improve from 1.18403

Epoch 00017: val_loss did not improve from 1.18403

Epoch 00018: val_loss did not improve from 1.18403

Epoch 00019: val

In [ ]:
! ls model_2

checkpoint_best.hdf5


In [ ]:
model = tf.keras.models.load_model("model_2/checkpoint_best.hdf5")
loss, acc = model.evaluate(X_test, y_test)
print(f"Accuracy of restored model {acc*100 :.2f}%")

5/5 [==============================] - 0s 3ms/step - loss: 1.1840 - accuracy: 0.3961
Accuracy of restored model 39.61%


Сохранять один файл или несколько -- решать вам, это зависит от задач и принятых практик ведения экспериментов, например, в каких-то случаях может пригодится сравнить предсказания нескольких последних моделей.


### Продолжение процесса обучения

In [ ]:
# мы можем продолжить процесс обучения, обратите внимание на initial_epoch
model = tf.keras.models.load_model("model_2/checkpoint_best.hdf5")
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=300, batch_size=64, verbose=0, 
          callbacks=[checkpoint], initial_epoch=150)


Epoch 00151: val_loss did not improve from 1.18403

Epoch 00152: val_loss did not improve from 1.18403

Epoch 00153: val_loss did not improve from 1.18403

Epoch 00154: val_loss did not improve from 1.18403

Epoch 00155: val_loss did not improve from 1.18403

Epoch 00156: val_loss did not improve from 1.18403

Epoch 00157: val_loss did not improve from 1.18403

Epoch 00158: val_loss did not improve from 1.18403

Epoch 00159: val_loss did not improve from 1.18403

Epoch 00160: val_loss did not improve from 1.18403

Epoch 00161: val_loss did not improve from 1.18403

Epoch 00162: val_loss did not improve from 1.18403

Epoch 00163: val_loss did not improve from 1.18403

Epoch 00164: val_loss did not improve from 1.18403

Epoch 00165: val_loss did not improve from 1.18403

Epoch 00166: val_loss did not improve from 1.18403

Epoch 00167: val_loss did not improve from 1.18403

Epoch 00168: val_loss did not improve from 1.18403

Epoch 00169: val_loss did not improve from 1.18403

Epoch 00170

## Заключение

В этом уроке мы узнали как сохранять прогресс обучения и использовать полученные модели. 

Следующий этап -- [домашнее задание](https://colab.research.google.com/drive/1G6iHt4PUVrjSHQ80wnCYm3gOtx_Gl1gE), в котором вы примените все полученные знания из этого модуля. Вы снова вернетесь к реализации классификатора известных вам рукописных цифр, но в этот раз сделаете это с помощью Tensorflow.